import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization,Input,ReLU,add,MaxPool1D,GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score,mean_absolute_error
from tensorflow.keras import callbacks,Model,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=900
cls_num=5
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Worms0(i)

os.chdir(cwd)
clses_lens=np.array([])
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (77, 901)
first_test_shape= (181, 901)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [33 13 10 13  8]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -4.8873
first_train_sample=
 [ 1.        -0.77859   -0.74444   -0.72546   -0.70269   -0.71408
 -0.72167   -0.73305   -0.71787   -0.73685   -0.76341   -0.77479
 -0.76341   -0.74823   -0.72546   -0.73305   -0.73305   -0.74064
 -0.74444   -0.76341   -0.7672    -0.76341   -0.75582   -0.74823
 -0.74823   -0.75582   -0.74823   -0.75962   -0.75203   -0.74823
 -0.74823   -0.73305   -0.73305   -0.74064   -0.771     -0.80895
 -0.83931   -0.86587   -0.87725   -0.87346   -0.919     -0.86207
 -0.86207   -0.85069   -0.82792   -0.85069   -0.78238   -0.7672
 -0.74444   -0.72546   -0.75203   -0.6951    -0.6951    -0.72167
 -0.72926   -0.74444   -0.74444   -0.79756   -0.75203   -0.67233
 -0.64577   -0.67613   -0.65716   -0.61541   -0.63818   -0.63439
 -0.59264   -0.61162   -0.5547    -0.42567   -0.30044   -0.2

1 33	1 33	2 13	3 10	4 13	

In [3]:
np.shape(ecg)

(102, 901)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 33	2 13	3 10	4 13	5 8	
 max = 33


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (33, 901)
Up to class  2 train shape =  (46, 901)
Up to class  3 train shape =  (56, 901)
Up to class  4 train shape =  (69, 901)
Up to class  5 train shape =  (77, 901)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 33 13 10 13  8]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 5., 5., 5., 5., 5., 5., 5., 5.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  4.1375
min magnitude class 1  =  -4.037800000000001
after normalizing >>
max magnitude class 1  =  0.9870973424048264
min magnitude class 1  =  -0.8129555012440277

 cls 2  >> 
max magnitude class 2  =  4.8591000000000015
min magnitude class 2  =  -4.3114
after normalizing >>
max magnitude class 2  =  1.1459805799590468
min magnitude class 2  =  -0.8731972609375345

 cls 3  >> 
max magnitude class 3  =  4.263200000000001
min magnitude class 3  =  -3.2160000000000006
after normalizing >>
max magnitude class 3  =  1.014774203492085
min magnitude class 3  =  -0.6320100402932824

 cls 4  >> 
max magnitude class 4  =  2.804300000000001
min magnitude class 4  =  -3.1806000000000005
after normalizing >>
max magnitude class 4  =  0.6935508730211157
min magnitude class 4  =  -0.6242156020873243

 cls 5  >> 
max magnitude class 5  =  2.7693
min magnitude class 5  =  -3.8073000000000006
after normalizing >>
max magnitude class 5  =  0.6858445075632471
min ma

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.8731972609375345
max tst = 1.1459805799590468


In [14]:
len(wndws_test)

181

In [15]:
np.shape(wndws_test)

(181, 901)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5], dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(33, 901)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (77, 900)
y_train =>  (77,)
X_test  =>  (181, 900)
y_test  =>  (181,)
X_valid  =>  (77, 900)
y_valid  =>  (77,)


In [22]:
if min(y_train)==0:
    print(' class was started from 0=Zero ')
else:
    print('!!!!!! class Should start from 0=Zero !!!!!!')
    print("!!!!!!   Reduce 1 From Class Number    !!!!!")
    y_train=np.array(y_train-1)
    y_test=np.array(y_test-1)
    y_valid=np.array(y_valid-1)

!!!!!! class Should start from 0=Zero !!!!!!
!!!!!!   Reduce 1 From Class Number    !!!!!


In [23]:
y_train_cat=to_categorical(y_train)#[:,1:]
y_valid_cat=to_categorical(y_valid)#[:,1:]
y_test_cat=to_categorical(y_test)#[:,1:]


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

<h1><center><div style="direction:rtl;font-family:B Nazanin">Network</div></center></h1>

In [24]:
# Import ResNet1D Module
#from ResNet_1DCNN import ResNet
#from ResNeXt_1DCNN import ResNeXt as ResNet
from SE_ResNeXt_1DCNN import SEResNeXt as ResNet

"Configurations for ResNet in Regression Mode"
length = X_train.shape[1]   # Number of Features (or length of the signal)
model_width = 128           # Number of Filter or Kernel in the Input Layer
num_channel = 1             # Number of Input Channels
problem_type = 'Classification' # Regression or Classification
output_number = cls_num     # Number of Outputs in the Regression Mode

Regression_Model = ResNet(length, num_channel, model_width, problem_type=problem_type, output_nums=output_number).SEResNeXt152() # Build Model
# ResNet Models supported: ResNet18, ResNet34, ResNet50, ResNet101, ResNet152, 
# ResNext Models supported: ResNeXt18, ResNeXt34, ResNeXt50, ResNeXt101, ResNeXt152, 
# SEResNext Models supported: SEResNeXt18, SEResNeXt34, SEResNeXt50, SEResNeXt101, SEResNeXt152, 
#Regression_Model.compile(loss='mae', optimizer='adam', metrics= ['mse']) # Compile Model
Regression_Model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# Here, Model validation metric is set as Mean Squared Error or MSE

#Regression_Model.summary() # Summary of the Model

# Early Stopping and Model_Checkpoints are optional parameters
# Early Stopping is to stop the training based on certain condition set by the user
# Model Checkpoint is to save a model in a directory based on certain conditions so that it can be used later for Transfer Learning or avoiding retraining
earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 100, restore_best_weights = True)
#callbacks = [EarlyStopping(monitor='val_loss', patience=30, mode='min'), ModelCheckpoint('Saved_Model.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
history = Regression_Model.fit(X_train, y_train_cat, epochs=500, batch_size=btch, verbose=1, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping]) #,validation_split=0.2,callbacks =[callbacks]
# Save 'History' of the model for model performance analysis performed later

# Preictions from the Test Set from the Trained Model
Predictions = Regression_Model.predict(X_test, verbose=1)
print(Predictions.shape)

# Error of the prediction, one of many evaluation metrics
# Using Mean Absolute Error (MAE) in this case as a sample
Error = mean_absolute_error(y_test_cat, Predictions)
print(f"MAE: {Error}")

Epoch 1/500
4/4 [==============================] - 153s 6s/step - loss: 4.1938 - accuracy: 0.2727 - val_loss: 1700045949675067408384.0000 - val_accuracy: 0.1688
Epoch 2/500
4/4 [==============================] - 6s 2s/step - loss: 2.5952 - accuracy: 0.4286 - val_loss: 1875606114774154215424.0000 - val_accuracy: 0.1688
Epoch 3/500
4/4 [==============================] - 6s 2s/step - loss: 1.4179 - accuracy: 0.4545 - val_loss: 88213756323817324544.0000 - val_accuracy: 0.1688
Epoch 4/500
4/4 [==============================] - 6s 2s/step - loss: 1.4474 - accuracy: 0.4416 - val_loss: 12806707919567454208.0000 - val_accuracy: 0.1688
Epoch 5/500
4/4 [==============================] - 6s 2s/step - loss: 2.3959 - accuracy: 0.4416 - val_loss: 582178899962626048.0000 - val_accuracy: 0.1688
Epoch 6/500
4/4 [==============================] - 6s 2s/step - loss: 1.9089 - accuracy: 0.4026 - val_loss: 35729837230391296.0000 - val_accuracy: 0.1688
Epoch 7/500
4/4 [==============================] - 6s 2s/

Epoch 58/500
4/4 [==============================] - 6s 2s/step - loss: 1.4966 - accuracy: 0.4416 - val_loss: 1.4489 - val_accuracy: 0.4416
Epoch 59/500
4/4 [==============================] - 6s 2s/step - loss: 1.3859 - accuracy: 0.4935 - val_loss: 1.4696 - val_accuracy: 0.4416
Epoch 60/500
4/4 [==============================] - 6s 2s/step - loss: 1.5719 - accuracy: 0.4416 - val_loss: 4.3682 - val_accuracy: 0.1688
Epoch 61/500
4/4 [==============================] - 6s 2s/step - loss: 1.4057 - accuracy: 0.4805 - val_loss: 1.9096 - val_accuracy: 0.1688
Epoch 62/500
4/4 [==============================] - 6s 2s/step - loss: 1.3534 - accuracy: 0.4935 - val_loss: 1.4739 - val_accuracy: 0.4416
Epoch 63/500
4/4 [==============================] - 6s 2s/step - loss: 1.3908 - accuracy: 0.4805 - val_loss: 1.4732 - val_accuracy: 0.4416
Epoch 64/500
4/4 [==============================] - 6s 2s/step - loss: 1.5901 - accuracy: 0.4545 - val_loss: 1.4535 - val_accuracy: 0.4416
Epoch 65/500
4/4 [=========

4/4 [==============================] - 6s 2s/step - loss: 1.6166 - accuracy: 0.3896 - val_loss: 3769628.2500 - val_accuracy: 0.1688
Epoch 116/500
4/4 [==============================] - 6s 2s/step - loss: 1.4753 - accuracy: 0.4026 - val_loss: 4806315.0000 - val_accuracy: 0.2338
Epoch 117/500
4/4 [==============================] - 6s 2s/step - loss: 1.4429 - accuracy: 0.4156 - val_loss: 107576048.0000 - val_accuracy: 0.2338
Epoch 118/500
4/4 [==============================] - 6s 2s/step - loss: 1.4755 - accuracy: 0.4416 - val_loss: 156664240.0000 - val_accuracy: 0.1688
Epoch 119/500
4/4 [==============================] - 6s 2s/step - loss: 1.5625 - accuracy: 0.4026 - val_loss: 5100441.0000 - val_accuracy: 0.1818
Epoch 120/500
4/4 [==============================] - 6s 2s/step - loss: 1.5009 - accuracy: 0.4545 - val_loss: 69238568.0000 - val_accuracy: 0.2208
Epoch 121/500
4/4 [==============================] - 6s 2s/step - loss: 1.4113 - accuracy: 0.4675 - val_loss: 17966728.0000 - val_acc

In [25]:
predict_x=Regression_Model.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                 # function from augment.py to remove 0 index predictions

#y_pred_crs = model_Res.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

6/6 [==============================] - 2s 273ms/step
array([[73,  3,  0,  0,  0],
       [26,  5,  0,  0,  0],
       [24,  1,  0,  0,  0],
       [32,  0,  0,  0,  0],
       [16,  1,  0,  0,  0]], dtype=int64)
              precision    recall  f1-score   support

           0       0.43      0.96      0.59        76
           1       0.50      0.16      0.24        31
           2       0.00      0.00      0.00        25
           3       0.00      0.00      0.00        32
           4       0.00      0.00      0.00        17

    accuracy                           0.43       181
   macro avg       0.19      0.22      0.17       181
weighted avg       0.26      0.43      0.29       181



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
